# _k_-nächste-Nachbarn-Klassifikation mit Scikit-learn

Der _k_-nächste-Nachbarn-Algorithmus ist ein Klassifikationsverfahren, das eine neue Beobachtung der Klasse zuordnet, zu der die meisten ihrer _k_ Nachbarn gehören.

## Vorbereitung des Datensatzes

<a href="https://seaborn.pydata.org/">_Seaborn_</a> ist eigentlich ein Python-Modul zur Datenvisualisierung. Es bietet aber die Funktion <a href="https://seaborn.pydata.org/generated/seaborn.load_dataset.html">_load_dataset()_</a> an, mit deren Hilfe man eine kleine Anzahl an Referenzdatensätzen aus einem Online-Repository laden kann.

In [ ]:
# Importiere die nötigen Module
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Im Folgenden arbeiten wir mit dem <a href="https://allisonhorst.github.io/palmerpenguins/">_Palmer Archipelago (Antarctica) penguin_</a> Datensatz. Dieser Datensatz enthält Informationen über 344 Pinguine, die auf drei Inseln des Palmer-Archipels in der Antarktis beheimatet sind. Die Merkmale des Datensatzes enthalten verschiedene Messdaten, die über die Pinguine gesammelt wurden, wie Länge und Höhe des Schnabels, Länge der Flosse und die Körpermasse. Das Zielmerkmal beschreibt die entsprechende Pinguin-Art. Dabei gehören die Beobachtungen bzw. die Pinguine im Datensatz zu einer der drei Arten, _Adelie_, _Chinstrap_ (deutsch:  Zügelpinguin) und _Gentoo_ (deutsch: Eselspinguin).

In [ ]:
# Lade den Palmer Archipelago (Antarctica) penguin Datensatz
penguins = sns.load_dataset("penguins")

# Gebe die ersten fünf Zeilen des Datensatzes mit den
# Bezeichnungen der Merkmale aus
print(penguins.head())

Der Datensatz wird in der Datenstruktur _DataFrame_ geladen. Diese Datenstruktur wandeln wir mit der Funktion <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html">_to_numpy()_</a> in ein _NumPy_-Array um. Für die weitere Analyse werden nur die Merkmale Schnabellänge, Schnabelhöhe, Flossenlänge und die Körpermasse in die Datenmatrix und die entsprechende Pinguin-Art in das Zielarray gespeichert.

In [ ]:
# Wandle das DataFrame in ein NumPy-Array um
penguins_arr = penguins.to_numpy()

In [ ]:
# Speichere das dritte (Schnabellänge), das vierte (Schnabelhöhe),
# das fünfte (Flossenlänge) und das sechste (Körpermasse)
# Merkmal in die Datenmatrix
X_raw = np.array(penguins_arr[:,2:6]).astype(float)

# Speichere das erste Merkmal als Zielmerkmal
y_raw = penguins_arr[:,0]

Da zwei Beobachtungen fehlende Werte enthalten, werden diese aus dem Datensatz entfernt. So verbleiben noch 342 Beobachtungen im Datensatz.

In [ ]:
# Lösche alle Zeilen aus der Datenmatrix, die
# fehlende Wert enthalten
X = X_raw[~np.isnan(X_raw).any(axis=1)]
y = y_raw[~np.isnan(X_raw).any(axis=1)]

Die Beobachtungen werden durch ihre Werte für die Schnabellänge, Schnabelhöhe und die Flossenlänge in einem 3-dimensionalen Diagramm grafisch dargestellt. Die Zugehörigkeit zur Pinguin-Art wird durch unterschiedliche Farben der Datenpunkte markiert.

In [ ]:
pyo.init_notebook_mode()
fig1 = px.scatter_3d(x=X[:,0], y=X[:,1], z=X[:,2], color=y)
fig1.show()

Bevor ein _k_-NN-Modell erstellt wird, wird die Datenmatrix in eine Trainings- und eine Testmenge zerlegt. Dafür kann die Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">_train\_test\_split()_</a> des Untermoduls von _Scikit-learn_ _Model Selection_ genutzt werden. Diese Funktion erwartet als Eingabe die Datenmatrix mit dem entsprechenden Zielmerkmal. In den Voreinstellungen der Funktion ist festgelegt, dass die Beobachtungen vor der Zerlegung gemischt werden. Für diesen Datensatz ist es besonders wichtig, diese Voreinstellung beizubehalten, weil die Beobachtungen in der ursprünglichen Datenmatrix nach dem Zielmerkmal sortiert sind. Würden wir auf das Mischen verzichten, wäre die resultierende Trainingsmenge für die gesamte Datenmenge nicht repräsentativ. Mit der Wertzuweisung des Parameters _random\_state_ stellen wir aber sicher, dass die Beobachtungen im Datensatz auf eine bestimmte Weise durchmischt werden. Damit sichern wir auch die Reproduzierbarkeit unserer Zerlegung.

In [ ]:
# Teile den Datensatz in die Trainings- und die Testmenge auf
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=45)

## _k_-nächste-Nachbarn-Modell

Ein _k_-nächste-Nachbar-Modell wird mit der Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html">_KNeighborsClassifier()_</a> des _Scikit-learn_-Untermoduls <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.neighbors">_Nearest Neighbors_</a> erstellt. Das Anpassen des Modells an die Trainingsdaten erfolgt mit der Funktion _fit()_. Bei diesem Modell findet kein richtiges Training statt. Stattdessen werden die Beobachtungen der Trainingsmenge intern in einer speziellen Datenstruktur gespeichert, die das schnelle Finden der _k_ nächsten Nachbarn ermöglicht.

In [ ]:
# In der Variable k wird die Nachbaranzahl gespeichert
k = 3

# Erstelle das knn-Modell
knn_classifier = neighbors.KNeighborsClassifier(n_neighbors=k)

# Trainiere das Modell mit der Trainingsmenge
knn_classifier.fit(X_train, y_train)

Mit der Funktion _predict()_ werden die Vorhersagen für alle Beobachtungen der Testmenge getroffen.

In [ ]:
# Erstelle die Vorhersage für die Testmenge
y_pred = knn_classifier.predict(X_test)

Um sehen zu können, wie gut das _k_-nächste-Nachbarn-Modell die Klasse für neue Beobachtungen schätzen kann, geben wir exemplarisch für die erste Beobachtung der Testmenge die tatsächliche und die durch das _k_-NN Klassifikationsmodell geschätzte Klasse bzw. Pinguin-Art aus.

In [ ]:
# Gebe den tatsächlichen Wert des Zielmerkmals für
# die erste Beobachtung der Testmenge aus
print("Die tatsächliche Pinguin-Art der "
      "ersten Beobachtung aus der Testmenge:", y_test[0])

# Gebe den geschätzten Wert des Zielmerkmals für
# die erste Beobachtung der Testmenge aus
print("Der geschätzte Pinguin-Art der "
      "ersten Beobachtung aus der Testmenge:", y_pred[0])

Um die Leistung eines Klassifikationsmodells zu bewerten, reicht es normalerweise nicht aus, die tatsächliche und die durch das Klassifikationsmodell geschätzte Klasse für eine Beobachtung miteinander zu vergleichen. Vielmehr werden diese beiden Werte für alle Beobachtungen der Testmenge miteinander verglichen und zu einem Gütemaß zusammengefasst. Eins dieser Gütemaße ist _Accuracy_, die auf Deutsch auch _Genauigkeit_ oder _Korrektklassifizierungsrate_ genannt wird. _Accuracy_ misst die Anzahl der richtigen Vorhersagen im Verhältnis zur Gesamtzahl aller Vorhersagen. Im Untermodul <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics">_Metrics_</a> von _Scikit-learn_ steht die Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html">_accuracy\_score()_</a> für die Berechnung der _Accuracy_ zur Verfügung. Diese Funktion erwartet zwei 1-dimensionale Arrays als Eingabe. Dabei soll das erste Array die tatsächlichen und das zweite Array die geschätzten Klassen der Beobachtungen enthalten. Als Ergebnis liefert die Funktion den Genauigkeitswert für die Schätzung.

In [ ]:
# Berechne die Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

Zum Schluss werden noch die Beobachtungen der Testmenge in einem 3-dimensionalen Diagramm grafisch dargestellt. Dafür werden wieder nur die ersten drei Merkmale, Schnabellänge, Schnabelhöhe und die Flossenlänge, verwendet. Die Pinguin-Arten der Beobachtungen werden durch unterschiedliche Farben markiert. Dabei werden durch die Diamanten die tatsächlichen und durch Kreuze die geschätzten Klassen der Beobachtungen abgebildet.

In [ ]:
# Visualisiere die tatsächliche Zugehörigkeit als Diamant
# und die geschätzte als Kreuz
fig3 = px.scatter_3d(x=X_test[:,0], y=X_test[:,1], z=X_test[:,2],
                     color=y_test, opacity=0.8,
                     symbol_sequence=["diamond"])
fig4 = px.scatter_3d(x=X_test[:,0], y=X_test[:,1], z=X_test[:,2],
                     color=y_pred, symbol_sequence=["cross"])
fig5 = go.Figure(data=fig3.data + fig4.data)
fig5.show()